# BE Smart Cities
The purpose of this project is to design a house with two rooms, and model and simulate the thermal behaviour of the building. We will study this in both steady state and dynamic simulation. 

## Modeling of the rooms

The layout we chose for our rooms is the following : 

[insert rooms plan here]

## Hypothesis

For simplicity of the model, we make the following hypothesis : 
 - The thermal transfers are linear, and the radiation is approximated by a coefficient close to convection
 - Length and height of the walls are considered to be 10 times greater than the width. Therefore we consider that the thermal transfer is unidirectional, and temperature is uniform on every surface.
 - We only consider the exchanges between vertical walls, assuming the floor and ceiling are adiabatic, such as the two walls without door and windows
 - We consider the indoor surface equal to the outdoor surface
 


## Thermal model 

### Main parameters

The first step was to constitue the thermal model of the rooms, using several elements : 

- The outer walls are made out of two materials : concrete and insulation 
Their dimensions are 6 x 2.5 x 0.16 m. 

- The inner wall is made out of concrete only. 

- The windows are 0.6 x 0.8 x 0.02 m. Wall A is only glass (6 x 2.5 x 0.02 m). 

- The doors are 2.20 x 0.8 x 0.05 m.


In [1]:
import numpy as np
import pandas as pd
import dm4bem

L = 6
wall_height = 2.5
SA = L*wall_height       # m² surface area of the glass wall (wall A)
Swindow = 0.6*0.8 # For 1 window
Sdoor= 2.2*0.8 # For 1 door
SB_c = L*wall_height-Swindow
SC_c = L*wall_height-Swindow-Sdoor
SD_c = L* wall_height
SE_c = L*wall_height-Sdoor
SF_c = L* wall_height
SG_c = L*wall_height
S_c = S_i = SB_c + SC_c + SD_c + SE_c + SF_c + SG_c
S_g= SA + 2*Swindow
S_d = 2*Sdoor
width_wall=0.16
width_insulation=0.04
width_window=0.02
width_door=0.05


We filled in the thermo-physical properties of air, and of the 4 materials listed before.

In [2]:
air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
pd.DataFrame(air, index=['Air'])


concrete = {'Conductivity': 1.046,          # W/(m·K)
            'Density': 2300.0,              # kg/m³
            'Specific heat': 657,           # J/(kg⋅K)
            'Width': 0.16,                   # m
            'Surface': S_c}            # m²


insulation = {'Conductivity': 0.0325,        # W/(m·K)
              'Density': 128.0,              # kg/m³
              'Specific heat': 1093,        # J/(kg⋅K)
              'Width': 0.04,                # m
              'Surface': S_i}          # m²


glass = {'Conductivity': 1,               # W/(m·K)
         'Density': 2500,                   # kg/m³
         'Specific heat': 1210,             # J/(kg⋅K)
         'Width': 0.02,                     # m
         'Surface': S_g}                   # m²


door = {'Conductivity': 0.63,               # W/(m·K)
         'Density': 825,                   # kg/m³
         'Specific heat': 1210,             # J/(kg⋅K)
         'Width': 0.04,                     # m
         'Surface': S_d}                   # m²
        

Conventional values for the convection coefficients for indoor and outdoor convection in W/(m²⋅K) are:

In [3]:
h = pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  # W/(m²⋅K)

Advection in the rooms 

In [4]:
# ventilation flow rate
V = L*L*wall_height                 # m³, volume of air


ACH = 1                     # 1/h, air changes per hour
Va_dot = ACH / 3600 * (V)    # m³/s, air infiltration


# ventilation & advection
Gv = air['Density'] * air['Specific heat'] * Va_dot


# P-controler gain
# Kp = 1e4            # almost perfect controller Kp -> ∞
# Kp = 1e-3           # no controller Kp -> 0
Kp = 0

### Electrical model of the rooms

We then drew the equivalent eletrical schema of our room and numbered each node (from 0 to 18) and each resistance (from 0 to 25). We added the sun irradiation and internal irradiation (electrical devices, users of the rooms…). 


[insert schema here]


In [5]:
# temperature nodes
θ = ['θ0', 'θ1', 'θ2', 'θ3', 'θ4', 'θ5', 'θ6', 'θ7', 'θ8', 'θ9', 'θ10', 'θ11', 'θ12', 'θ13', 'θ14', 'θ15', 'θ16', 'θ17', 'θ18']


# flow-rate branches
q = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17', 'q18', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24', 'q25']


# temperature nodes
nθ = 19      # number of temperature nodes
θ = [f'θ{i}' for i in range(nθ)]


# flow-rate branches
nq = 26     # number of flow branches
q = [f'q{i}' for i in range(nq)]

### Computation of matrices
The next step was to compute every value of conductance for convection and conduction and fill the conductance diagonal matrix G, the capacity diagonal matrix C, the incidence matrix A, the temperature source vector b and the heat flow source vector f

In [16]:
G = np.array(np.hstack(
    [h['out']*SB_c,
     2 * concrete['Conductivity']*SB_c/width_wall, 2 * concrete['Conductivity']*SB_c/width_wall,
     2 * insulation['Conductivity']/width_insulation*SB_c, 2 * insulation['Conductivity']/width_insulation*SB_c,
     h['in']*SB_c,
     h['out']*SC_c,
     2 * concrete['Conductivity']/width_wall*SC_c, 2 * concrete['Conductivity']/width_wall*SC_c,
     2 * insulation['Conductivity']/width_insulation*SC_c, 2 * insulation['Conductivity']/width_insulation*SC_c,
     h['in']*SC_c,
     h['out']*SE_c,
     2 * concrete['Conductivity']/width_wall*SE_c, 2 * concrete['Conductivity']/width_wall*SE_c,
     2 * insulation['Conductivity']/width_insulation*SE_c, 2 * insulation['Conductivity']/width_insulation*SE_c,
     h['in']*SE_c,
     Gv + (h['out']+h['in'])*(SA+Sdoor) + glass['Conductivity']/width_window*SA + door['Conductivity']/width_door*Sdoor,
     Gv + (h['out']+h['in'])*(2*Swindow+Sdoor) + 2*(glass['Conductivity']/width_window*Swindow) + door['Conductivity']/width_door*Sdoor,
     Kp,
     h['in']*SG_c,
     2 * concrete['Conductivity']/width_wall*SG_c, 2 * concrete['Conductivity']/width_wall*SG_c,
     h['in']*SG_c,
     Kp]))


G_matrix=np.zeros((nq,nq))
for i in range (nq):
   G_matrix[i,i]=G[i]

pd.DataFrame(G, index=q)



# C=np.zeros((nθ,nθ))
# C[1,1] = concrete["Density"]*concrete["Specific heat"]*width_wall*SB_c
# C[3,3]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SB_c
# C[6,6]=concrete["Density"]*concrete["Specific heat"]*width_wall*SC_c
# C[8,8]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SC_c
# C[11,11] = concrete["Density"]*concrete["Specific heat"]*width_wall*SE_c
# C[13,13]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SE_c
# C[16,16]=concrete["Density"]*concrete["Specific heat"]*width_wall*SG_c

C=np.zeros(nθ)
neglect_air_glass = False

if neglect_air_glass:
    C[1] = concrete["Density"]*concrete["Specific heat"]*width_wall*SB_c
    C[3]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SB_c
    C[6]=concrete["Density"]*concrete["Specific heat"]*width_wall*SC_c
    C[8]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SC_c
    C[11] = concrete["Density"]*concrete["Specific heat"]*width_wall*SE_c
    C[13]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SE_c
    C[16]=concrete["Density"]*concrete["Specific heat"]*width_wall*SG_c
    
else:
    C[1] = concrete["Density"]*concrete["Specific heat"]*width_wall*SB_c
    C[3]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SB_c
    C[6]=concrete["Density"]*concrete["Specific heat"]*width_wall*SC_c
    C[8]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SC_c
    C[11] = concrete["Density"]*concrete["Specific heat"]*width_wall*SE_c
    C[13]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SE_c
    C[16]=concrete["Density"]*concrete["Specific heat"]*width_wall*SG_c
    

# pd.set_option("display.precision", 3)
pd.DataFrame(C, index=θ)



A=np.zeros((26,19))
for i in range (0,5):
   A[i,i]=1
   A[i+1, i]=-1
for i in range(5, 10):
   A[i+1, i] = 1
   A[i+2, i] = -1




for i in range(10, 15):
   A[i+2, i] = 1
   A[i+3, i] = -1




A[18,15]=-1
A[22,16]=1
A[23,16]=1
A[5,17]=1
A[11,17]=1
A[19,17]=1
A[20,17]=-1
A[21,17]=-1
A[17,18]=1
A[18,18]=1
A[24,18]=-1
A[25,18]=-1




pd.DataFrame(A, index=q, columns=θ)


#To is Text, input from the DB
b = pd.Series(['To', 0, 0, 0, 0, 0, 'To', 0, 0, 0, 0, 0, 'To', 0, 0, 0, 0, 0, 'To', 'To', 'Ti_sp', 0, 0, 0, 0, 'Ti_sp'],
             index=q)


# Φo - solar radiation absorbed by the outdoor surface of the wall, W;
# Φi - solar radiation absorbed by the indoor surface of the wall, W;
# Qa - auxiliary heat gains (i.e., occupants, electrical devices, etc.), W; A VERIFIER


f = pd.Series(['Φo', 0, 0, 0, 'Φi', 'Φo',0,0, 0, 'Φi', 'Φo', 0, 0, 0, 'Φi', 'Φo', 0, 'Qa', 'Qa'], index=θ)



y = np.zeros(nθ)         # nodes
y[[17]] = 1              # nodes (temperatures) of interest
y[[18]] = 1
pd.DataFrame(y, index=θ)


# thermal circuit
A = pd.DataFrame(A, index=q, columns=θ)
G = pd.Series(G, index=q)
C = pd.Series(C, index=θ)
b = pd.Series(b, index=q)
f = pd.Series(f, index=θ)
y = pd.Series(y, index=θ)


TC = {"A": A,
      "G": G,
      "C": C,
      "b": b,
      "f": f,
      "y": y}

In [7]:
# On part du dictionnaire TC déjà existant
A = TC["A"].copy()
G = TC["G"]
b = TC["b"]
C = TC["C"]
f = TC["f"]
y = TC["y"]

# On ajoute les colonnes G et b à la matrice A
A["G"] = G
A["b"] = b


# S'assurer que les colonnes de A sont bien définies
cols = list(A.columns)

# Ajouter la ligne 'C' (vecteur)
A.loc["C"] = pd.Series(C, index=cols).astype("object")

# Ajouter la ligne 'f' (texte)
A.loc["f"] = pd.Series(f, index=cols).astype("object")

# Ajouter la ligne 'y' (vecteur booléen ou 0/1)
A.loc["y"] = pd.Series(y, index=cols).astype("object")


# Sauvegarde en CSV
A.to_csv("modele_thermique.csv")


## Steady-state response

Steady-state analysis focuses on the behavior of a system after transients have settled, specifically when the system reaches a stable equilibrium at a constant input.

For this first resolution, we assume : 
The indoor air temperature is controlled or not (i.e., the building is in free running).

The heat capacity of air and of glass is neglected or not.

The time step is calculated from the eigenvalues or it is imposed at a value designated as Δt.

In [8]:
controller = False
neglect_air_glass_capacity = False
imposed_time_step = False
Δt = 498    # s, imposed time step

In [9]:
print('Matrices and vectors for thermal circuit from Figure 1') 
df = pd.read_csv('./modele_thermique.csv')
df.style.apply(lambda x: ['background-color: yellow'
                          if x.name in df.index[-3:] or c in df.columns[-2:]
                          else '' for c in df.columns], axis=1)

Matrices and vectors for thermal circuit from Figure 1


,Unnamed: 0,θ0,θ1,θ2,θ3,θ4,θ5,θ6,θ7,θ8,θ9,θ10,θ11,θ12,θ13,θ14,θ15,θ16,θ17,θ18,G,b
0,q0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,363.000000,To
1,q1,-1.0,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,189.849000,0
2,q2,0.0,-1.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,189.849000,0
3,q3,0.0,0.000000,-1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,23.595000,0
4,q4,0.0,0.000000,0.000000,-1.000000,1.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,23.595000,0
5,q5,0.0,0.000000,0.000000,0.000000,-1.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,116.160000,0
6,q6,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,319.000000,To
7,q7,0.0,0.000000,0.000000,0.000000,0.0,-1.0,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,166.837000,0
8,q8,0.0,0.000000,0.000000,0.000000,0.0,0.0,-1.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,166.837000,0
9,q9,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,-1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,20.735000,0


In [20]:
# MODEL
# =====
# Thermal circuit
TC = dm4bem.file2TC('./modele_thermique.csv', name='', auto_number=False)

# by default TC['G']['q20'] = 0, i.e. Kp -> 0, no controller (free-floating)
if controller:
    TC['G']['q20'] = 1e3        # Kp -> ∞, almost perfect controller
    TC['G']['q26'] = 1e3        # Kp -> ∞, almost perfect controller

if neglect_air_glass_capacity:
    C[1] = concrete["Density"]*concrete["Specific heat"]*width_wall*SB_c
    C[3]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SB_c
    C[6]=concrete["Density"]*concrete["Specific heat"]*width_wall*SC_c
    C[8]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SC_c
    C[11] = concrete["Density"]*concrete["Specific heat"]*width_wall*SE_c
    C[13]=insulation["Density"]*insulation["Specific heat"]*width_insulation*SE_c
    C[16]=concrete["Density"]*concrete["Specific heat"]*width_wall*SG_c

print('Matrices and vectors for thermal circuit from Figure 1') 
df = pd.read_csv('./modele_thermique.csv')
df.style.apply(lambda x: ['background-color: yellow'
                          if x.name in df.index[-3:] or c in df.columns[-2:]
                          else '' for c in df.columns], axis=1)

Matrices and vectors for thermal circuit from Figure 1


,Unnamed: 0,θ0,θ1,θ2,θ3,θ4,θ5,θ6,θ7,θ8,θ9,θ10,θ11,θ12,θ13,θ14,θ15,θ16,θ17,θ18,G,b
0,q0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,363.000000,To
1,q1,-1.0,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,189.849000,0
2,q2,0.0,-1.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,189.849000,0
3,q3,0.0,0.000000,-1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,23.595000,0
4,q4,0.0,0.000000,0.000000,-1.000000,1.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,23.595000,0
5,q5,0.0,0.000000,0.000000,0.000000,-1.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,1.0,0.0,116.160000,0
6,q6,0.0,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,319.000000,To
7,q7,0.0,0.000000,0.000000,0.000000,0.0,-1.0,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,166.837000,0
8,q8,0.0,0.000000,0.000000,0.000000,0.0,0.0,-1.000000,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,166.837000,0
9,q9,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,-1.000000,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,20.735000,0


In [21]:
# State-space
[As, Bs, Cs, Ds, us] = dm4bem.tc2ss(TC)

In [23]:
bss = np.zeros(nq)        # temperature sources b for steady state
bss[[0, 6, 12, 18, 19]] = 10      # outdoor temperature
bss[[20, 25]] = 20            # indoor set-point temperature

f = pd.Series(['Φo', 0, 0, 0, 'Φi', 'Φo',0,0, 0, 'Φi', 'Φo', 0, 0, 0, 'Φi', 'Φo', 0, 'Qa', 'Qa'], index=θ)

fss = np.zeros(nθ)         # flow-rate sources f for steady state
fss[[6]] = 1000

A = TC['A']
G = TC['G']
diag_G = pd.DataFrame(np.diag(G), index=G.index, columns=G.index)

θss = np.linalg.inv(A.T @ diag_G @ A) @ (A.T @ diag_G @ bss + fss)
print(f'θss = {np.around(θss, 2)} °C')




θss = [ 9.91  9.74  9.57  8.21  6.84 12.81 18.18 17.56 12.57  7.58  9.76  9.3
  8.84  5.13  1.42 -9.33  0.    6.57  0.67] °C
